In [1]:
import pandas as pd
from string import punctuation
import string
from collections import Counter 
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import re
import nltk

In [3]:
#read data
data = pd.read_csv('data/GOOGLESCRAPERESULT.csv')
det = data['DETAILS_MEDS'].tolist()

In [3]:
def removeDate(str):
    temp = re.sub('^[0-9]{1,2}[ \w]{4}[ 0-9]{5}', ' ', str)
    return temp

In [4]:
def splitText(s):
    c = s.count('...')
    return s.split("...", c)

In [5]:
res = []
for item in det:
    res.append(splitText(item))
    
res = [item for sublist in res for item in sublist]

In [6]:
def removePunct(s):
    symb = [';', '?', '!', '&', '\n', '\xa0', "·", '|', '(', ')', '@', '/', ':', ' x ', ' X ', '+']
    for sy in symb:
        s = s.replace(sy, " ")
    return s

In [4]:
def nonASCII(s):
    printable = set(string.printable)
    return ''.join(filter(lambda x: x in printable, s))

In [7]:
def splitMeasure(s):
    temp = re.sub('([0-9]mg|[0-9]Mg|[0-9]MG|[0-9]ML|[0-9]Ml|[0-9]ml)', r' \1', re.sub('(mg+|Mg+|Ml+|ml+|MG+|ML+)', r' \1', s)).split()
    res = ' '.join(temp).replace('mg.', 'mg. ')
    return res

In [8]:
#if has lot of periods replace with comma. (in case confused with decimal)
def find(s):
    temp = s.count('.')
    return temp

def replacePeriod(s):
    if find(s) > 2:
        s = s.replace('.', ',')
    return s

In [9]:
def splitSentence(s):
    sent_text = nltk.sent_tokenize(s)
    return sent_text

In [10]:
def preprocess1(s):
    temp = splitSentence(replacePeriod(splitMeasure(nonASCII(removePunct(removeDate(s))))))
    return temp

In [11]:
res1 = []
for item in res:
    res1.append(preprocess1(item))
flat_list = [item for sublist in res1 for item in sublist]

In [12]:
def cleanPunct(s):
    '''
    remove punct , and . except on decimal
    '''
    i = 0
    res = ""
    s = s+' '
    while i < len(s) :
        if (s[i] is ',' and s[i+1].isalpha()) or (s[i] is ',' and s[i-1].isalpha()):
            temp = " "
        elif (s[i] is '-' and s[i+1].isspace()) or (s[i] is '-' and s[i-1].isspace()):
            temp = " "
        elif (s[i] is '.' and s[i+1].isalpha()) or (s[i] is '.' and s[i-1].isalpha()):
            temp = " "
        else:
            temp = s[i]
        res = res+temp
        i+=1
    return res

In [13]:
def shortFor(s):
    result = []
    spl = s.split()
    for it in spl:
        if it.lower() == 'mg':
            it = 'miligram'
        elif it.lower() == 'ml':
            it = 'mililiter'
        elif it.lower() == 'mcg':
            it = 'mikrogram'
        elif it.lower() == 'gr':
            it = 'gram'
        elif it.lower() == 'iu' or it.lower() == 'ui':
            it = 'unit-internasional'
        result.append(it)
    return ' '.join(result)

In [14]:
def remove_extra_space(text):
        """ Make extra space into one space. """
        text_list = text.split(' ')
        text_list_temp = []

        for word in text_list:
            if word.strip():
                text_list_temp.append(word.strip())

        return ' '.join(text_list_temp)

In [15]:
def removeSentence(s):
    """
    sentences less than 4 words are removed
    """
    co = s.split()
    if len(co) > 3:
        return s

In [16]:
def preprocess2(s):
    temp = removeSentence(remove_extra_space(shortFor(cleanPunct(s))))
    return temp

In [17]:
res2 = []
temp = []
restemp = []
for item in flat_list:
    temp.append(splitSentence(item))
    
restemp = [item for sublist in temp for item in sublist]
for item in restemp:
    res2.append(preprocess2(item))
    
res2 = list(filter(None, res2)) 

In [18]:
len(res2)

17034

In [20]:
import csv
from itertools import zip_longest
d = [res2]
export_data = zip_longest(*d, fillvalue = '')
with open('Data_Preprocessed.csv', 'w', encoding="utf-8", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow((["Data_Preprocessed"]))
    wr.writerows(export_data)
myfile.close()

# FREQUENT WORD

In [18]:
# add = []
# for i in res2:
#     split_it = i.split()   
#     add.append(split_it) 
    
# flat_list = [item for sublist in add for item in sublist]
# Counter = Counter(flat_list) 
# most_occur = Counter.most_common(600) 
  
# pd.set_option('display.max_rows',None)
# # Calling DataFrame constructor on list 
# df = pd.DataFrame(most_occur) 
# li = []
# for i in most_occur:
#     if not i[0].isnumeric():
#         li.append(i[0])

In [14]:
def Stopword(s):
    read = pd.read_csv('Stopword_NER.csv')
    more_stopword = read['Stopword'].tolist()
    temp = []
    for i in s.split():
        if i.lower() not in more_stopword:
            temp.append(i)
    return ' '.join(temp)

In [15]:
res3 = []
count = 0
for item in fix:
    count +=1
    res3.append(Stopword(item))

In [26]:
len(res2)

17844

In [23]:
df = pd.read_csv("Data_Preprocessed.csv")
df['Stopword'] = res3
df.to_csv('Data_Preprocessed.csv')